# HW 4 CS 334
Hanna Meinikheim

9-10-2021

Logistic Regression

# Logistic Regression
---
Is it going to rain in Boise today? Create a Jupyter Notebook at that performs Logistic Regression using scikit-learn to predict the answer to this yes/no question.

- [ ] **Find some data.** You’re looking for a data set that has historic weather data for Boise, Idaho. Search for one yourself, that’s part of the assignment! There are many websites that offer APIs or searches that allow free downloads of a given time range and location. Select at least one year (maybe more) of recent weather data. A good data set will have things like temperature, barometric pressure, wind speed, etc. And of course, it must have some indication about whether it rained that day, usually in the form of inches of precipitation.
- [ ] **Pull this data into your project.** Download the data, and import it into your project as a file. Perhaps using Pandas and import it as a CSV file? You may have to create a new column indicating the label: `0 = no rain, 1 = rain`. This column is an important part of the training process. It’s not a feature, it’s a label for that given vector.
- [ ] **Use Logistic Regression.** You’re trying to answer the question, “Is it going to rain today in Boise?” Train a model that tries to predict if it’s going to rain on a given day. The “day” is a vector/row/array/sample of given features that you feed into the model to get a prediction. Remember, you need the same number of features (columns) for a given prediction that you used to train the model.
- [ ] **How good is your model?** Is it better than a wild guess? Document your findings in the notebook.
- [ ] **Push Jupyter Notebook to github**

### Finding the data
- I found out that Weather Underground no longer gives out free weather API keys as part of their program. [Weather Underground API GONE?](https://forum.pvoutput.org/t/weather-underground-api-gone/576)
- I submitted an order to NOAA and got the requested data. I just have to upload it to the Jupyter Notebook.

In [24]:
#import pandas
import pandas as pd
import numpy as np

col_names = ['station', 'name', 'date', 'awnd', 'pgtm', 'prcp', 'snow', 'snwd', 'tavg', 'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5', 'wt01', 'wt02', 'wt03', 'wt05', 'wt06', 'wt08']
# PRCP = Precipitation (tenths of mm) 
# SNWD = Snow depth (mm) 
# SNOW = Snowfall (mm)
# TAVG = Average temperature (tenths of degrees C)
# TMAX = Maximum temperature (tenths of degrees C)
# TMIN = Minimum temperature (tenths of degrees C)
# AWND = Average daily wind speed (tenths of meters per second)
# WDF2 = Direction of fastest 2-minute wind (degrees)
# WDF5 = Direction of fastest 5-second wind (degrees)
# WSF2 = Fastest 2-minute wind speed (tenths of meters per second)
# WSF5 = Fastest 5-second wind speed (tenths of meters per second)
# PGTM = PGTM = Peak gust time (hours and minutes, i.e., HHMM)
### the below weather types have values of `0 for not there or 1 for present`
# WT01 = Weather Type [Fog, ice fog, or freezing fog (may include heavy fog)]
# WT06 = Weather Type [Glaze or rime]
# WT05 = Weather Type [Hail (may include small hail)]
# WT02 = Weather Type [Heavy fog or heaving freezing fog (not always distinguished from fog)]
# WT08 = Weather Type [Smoke or haze]
# WT03 = Weather Type [Thunder]

# load dataset
weather = pd.read_csv("weatherdataNOAA.csv", header=None, names=col_names, skiprows=[0])

In [25]:
weather.head()

,station,name,date,awnd,pgtm,prcp,snow,snwd,tavg,tmax,...,wdf2,wdf5,wsf2,wsf5,wt01,wt02,wt03,wt05,wt06,wt08
0,USW00024131,"BOISE AIR TERMINAL, ID US",1/1/2018,2.68,NaN,0.00,0.0,2.0,26,35,...,140,130,10.1,12.1,1.0,NaN,NaN,NaN,NaN,1.0
1,USW00024131,"BOISE AIR TERMINAL, ID US",1/2/2018,8.05,NaN,0.00,0.0,1.2,26,34,...,140,140,19.9,25.1,1.0,NaN,NaN,NaN,NaN,1.0
2,USW00024131,"BOISE AIR TERMINAL, ID US",1/3/2018,2.91,NaN,0.00,0.0,1.2,28,35,...,160,150,14.1,19.0,1.0,NaN,NaN,NaN,NaN,1.0
3,USW00024131,"BOISE AIR TERMINAL, ID US",1/4/2018,2.01,NaN,0.00,0.0,1.2,26,36,...,230,280,8.1,14.1,1.0,NaN,NaN,NaN,NaN,1.0
4,USW00024131,"BOISE AIR TERMINAL, ID US",1/5/2018,2.46,NaN,0.01,0.0,1.2,31,41,...,310,300,15.0,19.0,1.0,NaN,NaN,NaN,NaN,1.0


In [26]:
# dropping columns 'station' and 'name' because all rows are collected at the same place
weather.drop('station', axis=1, inplace=True)
weather.drop('name',  axis=1, inplace=True)
weather.head()

,date,awnd,pgtm,prcp,snow,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5,wt01,wt02,wt03,wt05,wt06,wt08
0,1/1/2018,2.68,NaN,0.00,0.0,2.0,26,35,19,140,130,10.1,12.1,1.0,NaN,NaN,NaN,NaN,1.0
1,1/2/2018,8.05,NaN,0.00,0.0,1.2,26,34,18,140,140,19.9,25.1,1.0,NaN,NaN,NaN,NaN,1.0
2,1/3/2018,2.91,NaN,0.00,0.0,1.2,28,35,20,160,150,14.1,19.0,1.0,NaN,NaN,NaN,NaN,1.0
3,1/4/2018,2.01,NaN,0.00,0.0,1.2,26,36,19,230,280,8.1,14.1,1.0,NaN,NaN,NaN,NaN,1.0
4,1/5/2018,2.46,NaN,0.01,0.0,1.2,31,41,26,310,300,15.0,19.0,1.0,NaN,NaN,NaN,NaN,1.0


In [27]:
# df['is_rich_method1'] = np.where(df['salary']>=50, 'yes', 'no')
weather['rain'] = np.where(weather['prcp']>0, 1, 0)
weather.head()

,date,awnd,pgtm,prcp,snow,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5,wt01,wt02,wt03,wt05,wt06,wt08,rain
0,1/1/2018,2.68,NaN,0.00,0.0,2.0,26,35,19,140,130,10.1,12.1,1.0,NaN,NaN,NaN,NaN,1.0,0
1,1/2/2018,8.05,NaN,0.00,0.0,1.2,26,34,18,140,140,19.9,25.1,1.0,NaN,NaN,NaN,NaN,1.0,0
2,1/3/2018,2.91,NaN,0.00,0.0,1.2,28,35,20,160,150,14.1,19.0,1.0,NaN,NaN,NaN,NaN,1.0,0
3,1/4/2018,2.01,NaN,0.00,0.0,1.2,26,36,19,230,280,8.1,14.1,1.0,NaN,NaN,NaN,NaN,1.0,0
4,1/5/2018,2.46,NaN,0.01,0.0,1.2,31,41,26,310,300,15.0,19.0,1.0,NaN,NaN,NaN,NaN,1.0,1


In [29]:
# fillin NaN with 0
weather = weather.fillna(0)
weather.head()

,date,awnd,pgtm,prcp,snow,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5,wt01,wt02,wt03,wt05,wt06,wt08,rain
0,1/1/2018,2.68,0.0,0.00,0.0,2.0,26,35,19,140,130,10.1,12.1,1.0,0.0,0.0,0.0,0.0,1.0,0
1,1/2/2018,8.05,0.0,0.00,0.0,1.2,26,34,18,140,140,19.9,25.1,1.0,0.0,0.0,0.0,0.0,1.0,0
2,1/3/2018,2.91,0.0,0.00,0.0,1.2,28,35,20,160,150,14.1,19.0,1.0,0.0,0.0,0.0,0.0,1.0,0
3,1/4/2018,2.01,0.0,0.00,0.0,1.2,26,36,19,230,280,8.1,14.1,1.0,0.0,0.0,0.0,0.0,1.0,0
4,1/5/2018,2.46,0.0,0.01,0.0,1.2,31,41,26,310,300,15.0,19.0,1.0,0.0,0.0,0.0,0.0,1.0,1
